# Creating a web scraper for Zillow.com
This scraper will search all of Zillow's listings for both recently sold and currently available single family homes in a specific state/county.

## Helpful videos:
https://www.youtube.com/watch?v=a3Cuq2csLWk

## Helpful websites:
curl.trillworks.com

In [1]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np
